In [ ]:
# Import all libraries and modules
import pandas as pd
import numpy as np

# Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

## Transformation - SMOTE, Downsampling, Scaling
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids
from sklearn.svm import SVC
from sklearn.utils import resample 

## Deep Learning Keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

# Model Performance Evaluation
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve
from sklearn.metrics import precision_recall_curve, auc

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import time

print('Libraries imported successfully')


In [ ]:
# Reading MIT-BIH Arrhythmia Dataset as an example
df = pd.read_csv('/kaggle/input/MIT-BIH Arrhythmia Database.csv') 

unique_types = df['type'].unique()
print(unique_types)
print(df.shape)

# Create binary target variable 
df['type'].value_counts()
class_mapping_lambda = lambda x: 0 if x == 'N' else 1
df['label'] = df['type'].apply(class_mapping_lambda)

# Print the value counts of the new 'class' column
print(df['label'].value_counts())
df.drop(['type', 'record'], axis=1, inplace=True)

df.head(10)

# Preprocessing: Downscaling, train test splot and scaling of features

In [ ]:
# Separate features and target variable
X = df.drop(['label'], axis=1)  # Features
y = df['label']  # Target variable

# Split the dataset into 80% training20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save original training dataset for later reference
X_train_original = X_train
y_train_original = y_train

# Scaling of features 
scaler = preprocessing.MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(scaler.scale_)

# Downsampling 

In [ ]:
# Separate majority and minority classes in original data
majority_class = df[df['label'] == 0]
minority_class = df[df['label'] == 1]

# Downsample the majority class
downsampled_majority = resample(majority_class, 
                                replace=False,    # Sample without replacement
                                n_samples=len(minority_class),  # Match minority class size
                                random_state=42)  # Reproducible results

# Combine minority class with downsampled majority class
balanced_df = pd.concat([downsampled_majority, minority_class])

# Shuffle the dataset
balanced_df = balanced_df.sample(frac=1, random_state=42)

# Check the class distribution
print(balanced_df['label'].value_counts())

# Further downsample the balanced dataset
downsampled_balanced = resample(balanced_df, 
                         replace=False,    # Sample without replacement
                                n_samples=10000,  # Set desired size
                                random_state=42)  # Reproducible results

# Shuffle the downsampled dataset
downsampled_balanced = downsampled_balanced.sample(frac=1, random_state=42)

# Check the class distribution
print(downsampled_balanced['label'].value_counts())

# Splitting the dataset into features (X) and labels (y)
X_train_ds = downsampled_balanced.drop(columns=['label'])
y_train_ds = downsampled_balanced['label']

# Print the shapes of the resulting sets
print("Shape of X_train after downsampling:", X_train_ds.shape)
print("Shape of y_train after downsampling:", y_train_ds.shape)

X_train = X_train_ds
y_train = y_train_ds

# Deep Learning 

# Dense Neural Networks - Comparison of different activation functions

In [ ]:
# Define the list of activation functions to compare
activation_functions = ["relu", "sigmoid", "tanh",]

# Create a dictionary to store classification reports and confusion matrices
results_dnn = {}

# Iterate over each activation function
for activation_func in activation_functions:
    # Build the DNN model
    inputs = Input(shape=(32,), name="Input")
    dense1 = Dense(units=10, activation=activation_func, name="Dense_1")
    dense2 = Dense(units=8, activation=activation_func, name="Dense_2")
    dense3 = Dense(units=6, activation=activation_func, name="Dense_3")
    dense4 = Dense(units=2, activation="softmax", name="Dense_4")  # 2 output units for binary classification
    x = dense1(inputs)
    x = dense2(x)
    x = dense3(x)
    outputs = dense4(x)

    # Compile the model
    model_dnn = Model(inputs=inputs, outputs=outputs)
    model_dnn.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Train the model
    history = model_dnn.fit(X_train, y_train, epochs=500, batch_size=100, validation_split=0.1, verbose=0)

    # Predictions
    y_pred = model_dnn.predict(X_test)
    y_pred_class_dnn = np.argmax(y_pred, axis=1) 

    # Evaluate model performance
    report = classification_report(y_test, y_pred_class_dnn, output_dict=True)
    confusion_mat = confusion_matrix(y_test, y_pred_class_dnn)


    # Store results in the dictionary
    results_dnn[activation_func] = {}
    results_dnn[activation_func]['classification_report'] = report
    results_dnn[activation_func]['confusion_matrix'] = confusion_mat
    results_dnn[activation_func]['y_pred_class_dnn'] = y_pred_class_dnn
    
# Print and analyze the results
for activation_func, result in results_dnn.items():
    print(f"Activation Function: {activation_func}")
    print("Classification Report:")
    print(pd.DataFrame(result['classification_report']))
    print("Confusion Matrix:")
    print(result['confusion_matrix'])
    print("\n")


# Confusion Matirces DNNs

In [ ]:
# Plot confusion matrices
num_activation_functions = len(results_dnn)
num_cols = 3
num_rows = (num_activation_functions + num_cols - 1) // num_cols

plt.figure(figsize=(15, 5 * num_rows))

for idx, (activation_func, result) in enumerate(results_dnn.items()):
    plt.subplot(num_rows, num_cols, idx+1)
    sns.heatmap(result['confusion_matrix'], annot=True, cmap='Blues', fmt='g', cbar=False)
    plt.title(f'{activation_func}', fontsize=14)
    plt.xlabel('Predicted labels', fontsize=14)
    plt.ylabel('True labels', fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

plt.tight_layout()
plt.show()


# Artificial Neural Network - Comparison of different activation functions


In [ ]:
# Define activation functions to compare
activation_functions = ['relu', 'sigmoid', 'tanh']

# Initialize dictionary to store results
results_ann = {}

# Loop through activation functions and train models
for activation_func in activation_functions:
    # Create and train ANN model
    model_ann = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=4, activation=activation_func, input_shape=(32,)),
        tf.keras.layers.Dense(units=4, activation=activation_func),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    model_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Recall'])
    model_ann.fit(X_train, y_train, batch_size=10, epochs=500, validation_split=0.1, verbose=0)
    
    # Make predictions
    y_pred_ann = model_ann.predict(X_test)
    y_pred_class_ann = (y_pred_ann > 0.5).astype("int32")
    
    # Evaluate model performance
    report = classification_report(y_test, y_pred_class_ann, output_dict=True)
    confusion_mat = confusion_matrix(y_test, y_pred_class_ann)
    
    # Store results in the dictionary
    results_ann[activation_func] = {}
    results_ann[activation_func]['classification_report'] = report
    results_ann[activation_func]['confusion_matrix'] = confusion_mat
    results_ann[activation_func]['y_pred_class_ann'] = y_pred_class_ann

# Print the results
for activation_func, result in results_ann.items():
    print(f"Activation Function: {activation_func}")
    print("Classification Report:")
    print(pd.DataFrame(result['classification_report']))
    print("Confusion Matrix:")
    print(result['confusion_matrix'])
    print("\n")


# Confusion Matrices ANNs

In [ ]:
# Plot confusion matrices
num_activation_functions = len(results_ann)
num_cols = 3
num_rows = (num_activation_functions + num_cols - 1) // num_cols

plt.figure(figsize=(15, 5 * num_rows))

for idx, (activation_func, result) in enumerate(results_ann.items()):
    plt.subplot(num_rows, num_cols, idx+1)
    sns.heatmap(result['confusion_matrix'], annot=True, cmap='Blues', fmt='g', cbar=False)
    plt.title(f'{activation_func}', fontsize=14)
    plt.xlabel('Predicted labels', fontsize=14)
    plt.ylabel('True labels', fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

plt.tight_layout()
plt.show()


# Model Comparison

# ROC Curves

In [ ]:
# Define activation functions for DNN
activation_functions_dnn = ["relu", "sigmoid", "tanh"]

# Plot ROC curves for DNN
plt.figure(figsize=(10, 8))
for activation_func, result in results_dnn.items():
    fpr, tpr, _ = roc_curve(y_test, result["y_pred_class_dnn"])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'DNN {activation_func} (AUC = {roc_auc:.2f})')

# Define activation functions for ANN
activation_functions_ann = ["relu", "sigmoid", "tanh"]

# Plot ROC curves for ANN
for activation_func, result in results_ann.items():
    fpr, tpr, _ = roc_curve(y_test, result["y_pred_class_ann"])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'ANN {activation_func} (AUC = {roc_auc:.2f})', linestyle='dashed')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guessing')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC Curves for DNN and ANN', fontsize=14)
plt.legend(fontsize=14)
plt.show()


# Precision-Recall Curve

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

# Define activation functions for DNN
activation_functions_dnn = ["relu", "sigmoid", "tanh"]

# Plot precision-recall curves for DNN
plt.figure(figsize=(10, 8))
for activation_func, result in results_dnn.items():
    precision, recall, _ = precision_recall_curve(y_test, result["y_pred_class_dnn"])
    pr_auc = auc(recall, precision)
    plt.plot(recall, precision, label=f'DNN {activation_func} (AUC = {pr_auc:.2f})')

# Define activation functions for ANN
activation_functions_ann = ["relu", "sigmoid", "tanh"]

# Plot precision-recall curves for ANN
for activation_func, result in results_ann.items():
    precision, recall, _ = precision_recall_curve(y_test, result["y_pred_class_ann"])
    pr_auc = auc(recall, precision)
    plt.plot(recall, precision, label=f'ANN {activation_func} (AUC = {pr_auc:.2f})', linestyle='dashed')

plt.xlabel('Recall (True Positive Rate)', fontsize=14)
plt.ylabel('Precision', fontsize=14)
plt.title('Precision-Recall Curves for DNN and ANN', fontsize=14)
plt.legend(fontsize=14)
plt.show()



# Model refinement for DNN

In [ ]:
# MODEL 1 - Define the DNN architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(32,)))  # Input layer with ReLU activation
model.add(Dense(32, activation='relu'))  # Hidden layer with ReLU activation
model.add(Dropout(0.2))  # Dropout regularization to reduce overfitting
model.add(Dense(16, activation='relu'))  # Hidden layer with ReLU activation
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=100, validation_split=0.1, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype("int32")  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred_class))
print(confusion_matrix(y_test, y_pred_class))
y_pred_class_DNN2 = y_pred_class
y_pred_class_DNN2 = y_pred_class

In [ ]:
# MODEL 2 -  Define the DNN architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(32,)))  # Input layer with ReLU activation
model.add(Dense(64, activation='relu'))  # Hidden layer with ReLU activation
model.add(Dropout(0.3))  # Dropout regularization to reduce overfitting
model.add(Dense(32, activation='relu'))  # Hidden layer with ReLU activation
model.add(Dense(16, activation='relu'))  # Additional hidden layer with ReLU activation
model.add(Dense(8, activation='relu'))  # Additional hidden layer with ReLU activation
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=100, validation_split=0.1, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype("int32")  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred_class))
print(confusion_matrix(y_test, y_pred_class))
y_pred_class_DNN4 = y_pred_class
y_pred_class_DNN4 = y_pred_class

In [ ]:
# Define confusion matrices
conf_matrix_1 = np.array([[8639, 403], [26, 1001]]) # DNN Model 1 with downsampled 10 k dataset 

conf_matrix_2 = np.array([[8610, 432], [39, 988]]) # DNN Model 2 with downsampled 10 k dataset 

# Plot both confusion matrices side by side
plt.figure(figsize=(16, 6))

# Plot confusion matrix 1
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix_1, annot=True, cmap="Blues", fmt="d", annot_kws={"fontsize": 14})
plt.title("DNN Model 1 - downsampled dataset 10k", fontsize=16)
plt.xlabel("Predicted Labels", fontsize=14)
plt.ylabel("True Labels", fontsize=14)

# Plot confusion matrix 2
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_2, annot=True, cmap="Blues", fmt="d", annot_kws={"fontsize": 14})
plt.title("DNN Model 2 - downsampled dataset 10k", fontsize=16)
plt.xlabel("Predicted Labels", fontsize=14)
plt.ylabel("True Labels", fontsize=14)

plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()
